# Prepare Data

In [6]:
!pip3 install wikipedia-api


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [54]:
import wikipediaapi



def get_categorymembers(categorymembers, level=0, max_level=1, root_pages=None):
    if root_pages is None:
        root_pages = []

    for c in categorymembers.values():
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            get_categorymembers(c.categorymembers, level=level + 1, max_level=max_level, root_pages=root_pages)
        else:
            if c.title.startswith('Category'):
                try:
                    title  = c.title.split(":")[1]
                except:
                    continue
            else:
                title = c.title
            root_pages.append(title)
    return root_pages

def get_root_pages_from_category(category_name):
    cat = wiki_wiki.page(f"Category:{category_name}")
    return get_categorymembers(cat.categorymembers)

# Example usage:

API_KEY = 'aipi590 (hongxuan.li@duke.edu)'
wiki_wiki = wikipediaapi.Wikipedia(API_KEY, 'en', extract_format=wikipediaapi.ExtractFormat.WIKI)

domains = ["Statistics", "Artificial intelligence", "Computational mathematics", "Numerical analysis", "Applied mathematics", "Probability"]
root_pages = []
for domain in domains:
    root_page = get_root_pages_from_category(domain)
    root_pages += root_page

root_pages = set(root_pages)

# print(wiki_wiki.page("Random forest").text)

In [56]:
from mediawikiapi import MediaWikiAPI
mediawikiapi = MediaWikiAPI()

wiki_data = {"topic":[], "text":[]}
for page in root_pages:
   wiki_data["topic"].append(page)
   try:
      wiki_data["text"].append(mediawikiapi.page(page).content)
   except:
      wiki_data["text"].append(" ")
# wiki_data = {"topic":[], "text":[]}
# for page in root_pages:
#    wiki_data["topic"].append(page)
#    wiki_data["text"].append(wiki_wiki.page(page).text)


In [65]:
# import pandas as pd 
# wiki_df = pd.DataFrame(wiki_data)
# wiki_df = wiki_df.query("text.str.len() > 1")
# wiki_df = wiki_df.reset_index(drop=True)
# wiki_df.to_parquet('./data/wiki_ml_mediawiki.parquet')



In [234]:
import pandas as pd 

def find_noise_section(strings):
    for index, string in enumerate(strings):
        if string.startswith("See also.") or string.startswith("External links.") or string.startswith("References.") or string.startswith("Bibliography."):
            return index
    return -1


wiki_df = pd.read_parquet('./data/wiki_ml_mediawiki.parquet')

def clean_text(text):
    
    paragraphs = list(clean_markup(text, ignore_headers=False))
    
    # text = [paragraph for paragraph in paragraphs if len(paragraph) > 3]
    text = []
    for paragraph in paragraphs:
        if len(paragraph) > 5 or len(text) == 0:
            text.append(paragraph)
        else:
            text[-1] = text[-1] + " " + " ".join(paragraph.split(" "))
    text = "\n".join(text)
    
    index = find_noise_section(text.split("## "))
    if index!= -1:
        text = text.split("## ")[:index]
        return " ##".join(text)
    
    return text
    

wiki_df.text = wiki_df.text.apply(clean_text)
wiki_df["id"] = wiki_df.index
wiki_df.to_parquet('./data/wiki_ml_mediawiki_cleaned.parquet')

In [233]:
# wiki_df = pd.read_parquet('./data/wiki_ml_mediawiki.parquet')
# topic = "Artificial psychology"
# text = wiki_df.query("topic == @topic").text.values[0]
# text = list(clean_markup(text, ignore_headers=False))
# text = "\n".join(text)
# index = find_noise_section(text.split("## "))
# text

'Artificial psychology (AP) has had multiple meanings dating back to 19th century, with recent usage related to artificial intelligence (AI).\nIn 1999, Zhiliang Wang and Lun Xie presented a theory of artificial psychology based on artificial intelligence. They analyze human psychology using information science research methods and artificial intelligence research to probe deeper into the human mind.\n## Main Theory.\nDan Curtis (b. 1963) proposed AP is a theoretical discipline. The theory considers the situation when an artificial intelligence approaches the level of complexity where the intelligence meets two conditions:\nCondition I\nA: Makes all of its decisions autonomously\nB: Is capable of making decisions based on information that isNew\nAbstract\nIncompleteC: The artificial intelligence is capable of reprogramming itself based on the new data, allowing it to evolve.\nD: And is capable of resolving its own programming conflicts, even in the presence of incomplete data. This mean

In [222]:
  

text = wiki_df.iloc[[811],:].text.values[0]
# print(find_index_of_special_string(text.split("## ")))
# text.split("## ")
print(text)

In mathematics, a graph partition is the reduction of a graph to a smaller graph by partitioning its set of nodes into mutually exclusive groups. Edges of the original graph that cross between the groups will produce edges in the partitioned graph. If the number of resulting edges is small compared to the original graph, then the partitioned graph may be better suited for analysis and problem-solving than the original. Finding a partition that simplifies graph analysis is a hard problem, but one that has applications to scientific computing, VLSI circuit design, and task scheduling in multiprocessor computers, among others. Recently, the graph partition problem has gained importance due to its application for clustering and detection of cliques in social, pathological and biological networks. For a survey on recent trends in computational methods and applications see Buluc et al. (2013).
Two common examples of graph partitioning are minimum cut and maximum cut problems.
 ##Problem comp

In [236]:
"and implemented, e.g., in scikit-learn is spectral clustering".split()

['and',
 'implemented,',
 'e.g.,',
 'in',
 'scikit-learn',
 'is',
 'spectral',
 'clustering']

# Chunk Data

In [31]:
import pandas as pd
from tqdm import tqdm


def sliding_window(wiki_data, chunck_size = 20, step_size=10):
    chunks = []
    chunks_title =  []
    
    for i in tqdm(range(len(wiki_data))):
        title = wiki_data[i][0]
        text = wiki_data[i][1]
        words = text.split()
    
        for i in range(0, len(words) - chunck_size + 1, step_size):
            chunks.append(" ".join(words[i:i + chunck_size]))
            chunks_title.append(title)
    return chunks, chunks_title


wiki_data = pd.read_parquet('./data/wiki_ml_mediawiki_cleaned.parquet').values.tolist()

chunck_size = 50
step_size = 10
chunks, chunks_title = sliding_window(wiki_data, chunck_size, step_size)
len(chunks)

100%|██████████| 6858/6858 [00:01<00:00, 3686.69it/s]


843558

In [260]:
chunks[4]

'in which p 0 x is a non-zero constant. Among the most notable Appell sequences besides the trivial example {'

# Create Embeddings

In [27]:
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset
from tqdm import tqdm
import torch
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pad_sequence



class Encoder:
    def __init__(self, checkpoint, MAX_SEQ_LEN, device="cuda:0"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

        self.MAX_SEQ_LEN = MAX_SEQ_LEN

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=self.MAX_SEQ_LEN)
        return tokens.to(self.device)  

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=32):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader
        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings
    






Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# len(data.dataset[1]['input_ids'])
len(data.dataset[0]["attention_mask"])

35

In [28]:
# data = model.encode(chunks, show_progress_bar=False)
# for batch in data:
#     # print(batch)
#     continue

In [34]:
from time import time

MODEL_PATH = "lyeonii/bert-small"

MAX_SEQ_LEN = 50

device="cpu"

start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")
model = Encoder(MODEL_PATH, MAX_SEQ_LEN, device=device)

# # Get embeddings of prompts
# f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
# inputs = df.apply(f, axis=1).values # better results than prompt only
# prompt_embeddings = model.encode(inputs, show_progress_bar=False)


# # Get embeddings of chunks
chunck_embeddings = model.encode(chunks, show_progress_bar=True)

# # Search closest sentences in the wikipedia index 
# print(f"Loading faiss index, t={time() - start :.1f}s")
# faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# # faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

# print(f"Starting text search, t={time() - start :.1f}s")
# search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

# print(f"Starting context extraction, t={time() - start :.1f}s")
# dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
# for i in range(len(df)):
#     df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# # Free memory
# faiss_index.reset()
# del faiss_index, prompt_embeddings, model, dataset
# clean_memory()
# print(f"Context added, t={time() - start :.1f}s")

Starting prompt embedding, t=0.0s


  0%|          | 0/26362 [00:00<?, ?it/s]


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [ ]:
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display
# prepare template 
template = """Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

Question: {prompt}\n
A) {a}\n
B) {b}\n
C) {c}\n
D) {d}\n
E) {e}\n

### Answer: {answer}"""

prompt = PromptTemplate(template=template, input_variables=['prompt', 'a', 'b', 'c', 'd', 'e', 'answer'])

# display sample to see template
sample = train_dataset['train'][0]
display(Markdown(prompt.format(prompt=sample['prompt'], 
                               a=sample['A'], 
                               b=sample['B'], 
                               c=sample['C'], 
                               d=sample['D'], 
                               e=sample['E'], 
                               answer=sample['answer'])))

In [126]:
from datasets import load_dataset

dataset = load_dataset("lukaemon/mmlu", "machine_learning", split="test")
few_shots = load_dataset("lukaemon/mmlu", "machine_learning", split="train")

test_df = pd.DataFrame(dataset)
test_df

Reusing dataset mmlu (/Users/lihongxuan/.cache/huggingface/datasets/lukaemon___mmlu/machine_learning/1.0.0/134145dc2582b9a08b42d1f4b828f84a0066e9cc2e7dd8c1d83bee475746ecc3)
Reusing dataset mmlu (/Users/lihongxuan/.cache/huggingface/datasets/lukaemon___mmlu/machine_learning/1.0.0/134145dc2582b9a08b42d1f4b828f84a0066e9cc2e7dd8c1d83bee475746ecc3)


,input,A,B,C,D,target
0,Statement 1| RoBERTa pretrains on a corpus tha...,"True, True","False, False","True, False","False, True",C
1,"Statement 1| Support vector machines, like log...","True, True","False, False","True, False","False, True",B
2,A machine learning problem involves four attri...,12,24,48,72,D
3,"As of 2020, which architecture is best for cla...",convolutional networks,graph networks,fully connected networks,RBF networks,A
4,Statement 1| The log-likelihood of the data wi...,"True, True","False, False","True, False","False, True",B
...,...,...,...,...,...,...
106,Which of the following is true of a convolutio...,Convolving an image with $\begin{bmatrix}1 & 0...,Convolving an image with $\begin{bmatrix}0 & 0...,Convolving an image with $\begin{bmatrix}1 & 1...,Convolving an image with $\begin{bmatrix}0 & 0...,B
107,Which of the following is false?,Semantic segmentation models predict the class...,A bounding box with an IoU (intersection over ...,When a predicted bounding box does not corresp...,A bounding box with an IoU (intersection over ...,D
108,Which of the following is false?,The following fully connected network without ...,"Leaky ReLU $\max\{0.01x,x\}$ is convex.",A combination of ReLUs such as $ReLU(x) - ReLU...,The loss $\log \sigma(x)= -\log(1+e^{-x})$ is ...,C
109,We are training fully connected network with t...,111021,110010,111110,110011,A


# Encoder

In [ ]:
from collections.abc import Iterable
import pandas as pd
import blingfire as bf
from tqdm import tqdm

def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

Category:Social robots (id: ??, ns: 0)